# Neural Network Ejemplo

Construir una red neuronal de dos capas totalmente conectadas (también conocida como perceptrón multicapa) con TensorFlow.

## Neural Network

<img src="http://cs231n.github.io/assets/nn1/neural_net2.jpeg" alt="nn" style="width: 400px;"/>

## MNIST Descripción general del conjunto de datos

Este ejemplo utiliza dígitos manuscritos MNIST. El conjunto de datos contiene 60.000 ejemplos de formación y 10.000 ejemplos de pruebas. Los dígitos han sido normalizados y centrados en una imagen de tamaño fijo (28x28 píxeles) con valores de 0 a 1. Para simplificar, cada imagen ha sido aplanada y convertida a una matriz numérica 1-D de 784 características (28*28).

![MNIST Dataset](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)

Más información: http://yann.lecun.com/exdb/mnist/

In [1]:
from __future__ import print_function

# Importar datos MNIST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# Parametros
learning_rate = 0.1
num_steps = 500
batch_size = 128
display_step = 100

# Network Parametros
n_hidden_1 = 256 # Número de neuronas de la 1ª capa
n_hidden_2 = 256 # Número de neuronas de la 2ª capa
num_input = 784 # MNIST entrada de datos (forma img: 28*28)
num_classes = 10 # MNIST total clases (0-9 digitos)

# tf Entrada de gráficos
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [3]:
# Almacenar el peso y el sesgo de las capas
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [4]:
# Crear modelo
def neural_net(x):
    # Capa oculta totalmente conectada con 256 neuronas
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Capa oculta totalmente conectada con 256 neuronas
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Salida de una capa totalmente conectada con una neurona para cada clase
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [5]:
# Construir modelo
logits = neural_net(X)

# Definir la pérdida y el optimizador
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluar el modelo (con registros de pruebas, para deshabilitar la deserción escolar)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Inicializar las variables (es decir, asignar su valor por defecto)
init = tf.global_variables_initializer()

In [6]:
# Iniciar Entrenamiento
with tf.Session() as sess:

    # Ejecutar el inicializador
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Optimización de la ejecución (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calcular la pérdida y la precisión de los lotes
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calcular la precisión de las imágenes de prueba MNIST
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images,
                                      Y: mnist.test.labels}))

Step 1, Minibatch Loss= 13208.1406, Training Accuracy= 0.266
Step 100, Minibatch Loss= 462.8610, Training Accuracy= 0.867
Step 200, Minibatch Loss= 232.8298, Training Accuracy= 0.844
Step 300, Minibatch Loss= 85.2141, Training Accuracy= 0.891
Step 400, Minibatch Loss= 38.0552, Training Accuracy= 0.883
Step 500, Minibatch Loss= 55.3689, Training Accuracy= 0.867
Optimization Finished!
Testing Accuracy: 0.8729
